In [16]:
import gym
from gym.spaces import Discrete, MultiDiscrete
import numpy as np
import random as rand
import matplotlib.pyplot as plt
import math

In [17]:
class Grid(gym.Env):
    
    def __init__(self, **kwargs):        
    
        super().__init__()
        
        self.width = kwargs.get('width',8)
        self.height = kwargs.get('height',8)     
        
        self.action_space = gym.spaces.Discrete(5)
        self.observation_space = self.width*self.height #MultiDiscrete([self.width, self.height])
        
        self.max_x = self.width - 1
        self.max_y = self.height - 1
        
        self.episodes = 10000
        
        self.x = 0
        self.y = 0
        self.reward = -1
        
#         self.carolsx = rand.randint(0,7)
#         self.carolsy = rand.randint(0,7)

    def step(self, action):        
        state = 1    
        reward = -1       
        done = True
        info = {}
        return state, reward, done, info

    def reset(self):
        self.x = 0
        self.y = 0                
        return np.array([self.x,self.y])
  
    def render(self, mode):
        pass

In [ ]:
#from stable_baselines3.common.env_checker import check_env

In [10]:
from enum import IntEnum

class Actions(IntEnum):  
    Stay  = 0    
    North = 1
    East  = 2
    South = 3
    West  = 4

    def __str__(self): return self.name  

In [19]:
class GridforPaul( Grid ):

    metadata = {'render_modes': ['human']}
  
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
      
      #defining positions for the grid
        self.start = kwargs.get('start',[0,0])       
        self.end = kwargs.get('end',[self.max_x,self.max_y])        
      
        self.initial_pos = kwargs.get('initial_pos',self.start)  

      # initializing Paul's position
        self.x = self.initial_pos[0]
        self.y = self.initial_pos[1]
        
        #self.bank = self.width + 2
    
    ##Paul randomly chooses an action. 
        
    def choose_action(self):
        actions_dict = [Actions.Stay, Actions.North, Actions.East, Actions.South, Actions.West]
        value = rand.randint(0,4)
        action = actions_dict[value]
        return action
    
    ###see with each step if Paul runs into Carol and gets caught. 
      ###see with each step if Paul runs into Carol and gets caught. 
    
    
    def carolspicker(self, action):
        original_pos_x = self.x
        original_pos_y = self.y
        #values_ops = [0, 1, 2, 3, 4, 5]
        listed_actions = []
        for i in range(0, 5):
            #print(self.x, self.y)
            action_upd = Actions(i)
            #print(action_upd)
            new_val = self.take_action(action_upd)
            eDistance = math.dist([self.x, self.y], [3, 3])
            listed_actions.append(eDistance)
            self.x = original_pos_x
            self.y = original_pos_y
        carols_pick = min(listed_actions)
        index = listed_actions.index(carols_pick)
            #print(index)
        #print(listed_actions)
        action = Actions(index)
        #action = values_ops[index]
        #print(action)
        return action
    def carolschoice(self, action):
        #action = self.choose_actions()
        value = rand.choice([-1,1])
        #print(value)
        if value == 1:
            #print(action)
            #self.bank -=1
            return action
            
        else:
            action = self.carolspicker(action)
            #self.bank -=1
            #print(action)
            return action
        
    def checkcarolsloc(self, val1, val2):
        done = False
        if self.x == val1:
            if self.y == val2:
                reward = -100
                done = True
                print("You have been caught; game is over")
        return done
    
    ####actually moving on the grid
      
    def take_action(self, action):
#         actions_dict = [Actions.Stay, Actions.North, Actions.East, Actions.South, Actions.West]
#         value = rand.randint(0,5)
#         action = actions_dict[value]
#         print(action)
        
      # move in the direction of the specified action
        if action == Actions.North: 
            action = self.carolschoice(action)
            print(action)
            if action == Actions.North:
                self.y += 1
                print(self.x, self.y)
            else:
                self.y -= 1
                print(self.x, self.y)
        elif action == Actions.South: 
            action = self.carolschoice(action)
            print(action)
            if action == Actions.South:
                self.y += 1
                print(self.x, self.y)
            else:
                self.y -= 1
                print(self.x, self.y)
        elif action == Actions.West:
            action = self.carolschoice(action)
            print(action)
            if action == Actions.West:
                self.x -= 1
                print(self.x, self.y)
            else:
                self.x += 1
                print(self.x, self.y)
        elif action == Actions.East:
            action = self.carolschoice(action)
            print(action)
            if action == Actions.East:
                self.x += 1
                print(self.x, self.y)
            else:
                self.x -= 1
                print(self.x, self.y)
                
        elif action == Actions.Stay:
            return "No change has been made. The current coordinates are ({0}, {1}) ".format(self.x, self.y) 
        
        else:
            pass
        
        #bank -=1
      
      # staying on the grid until exit
        if self.x < 0: 
            self.x = 0
        if self.y < 0: 
            self.y = 0
        if self.x > self.max_x: 
            self.x = self.max_x
        if self.y > self.max_y: 
            self.y = self.max_y 
            
    ### seeing how Carol's choice impacts his actions and directions -> randomly chosen
            
        
    def step(self, action): 

      # take the action and update the position
        self.take_action(action)
        self.checkcarolsloc(3, 3)
        #self.carolschoice(action)
        obs = np.array([self.x,self.y])
        reward = 0
            
      # set the 'done' flag if we've reached the exit
        done = (self.x == self.end[0]) and (self.y == self.end[1])
      
      # get -1 reward for each step and -100 if you end up in Carol's spot
        if done:
            reward = 0
        elif self.checkcarolsloc(3,3):
            reward = -100
            done = True
            #return "The game is over"
        else:
            reward = 0
        info = {}
        return obs, reward, done, info     


#     def render(self, mode='human', action=0, reward=0 ):
#         if mode == 'human':
#             print(f"{Actions(action): <5}: ({self.x},{self.y}) reward = {reward}") 
#         else:
#             super().render(mode=mode) # just raise an exception

reward = 0 if done else -

In [20]:
env = GridforPaul()
env.reset()

array([0, 0])

In [ ]:
action = env.choose_action()

In [ ]:
env.bank

In [ ]:
env.action_space.sample()

In [ ]:
print(env.take_action(action))

## Starting the Q-Learning Process

In [21]:
#qtable = np.zeros((16, 4))

nb_states = env.observation_space  # = 64
nb_actions = env.action_space     # = 5
qtable = np.zeros((64,5))
print('Q-table =')
print(qtable)

Q-table =
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]


In [22]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams.update({'font.size': 17})

qtable = np.zeros((64,5))

# Parameters
episodes = 10000        # Total number of episodes
alpha = 0.8            # Learning rate
gamma = 0.9            # Discount factor

# List of outcomes to plot
outcomes = []

print('Q-table before training:')
print(qtable)

# Training
for _ in range(episodes):
    state = env.reset()
    done = False

    # By default, we consider our outcome to be a failure
    outcomes.append("Failure")

    # Until the agent gets stuck in a hole or reaches the goal, keep training it
    while not done:
        # Choose the action with the highest value in the current state
        if np.max(qtable[state]) > 0:
            action = np.argmax(qtable[state])

        # If there's no best action (only zeros), take a random one
        else:
            action = env.action_space.sample()
             
        # Implement this action and move the agent in the desired direction
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a)
        qtable[state, action] = qtable[state, action] + \
                                alpha * (reward + gamma * np.max(qtable[new_state]) - qtable[state, action])
        
        # Update our current state
        state = new_state

        if reward:
            outcomes[-1] = "Success"

print()
print('===========================================')
print('Q-table after training:')
print(qtable)

# Plot outcomes
plt.figure(figsize=(12, 5))
plt.xlabel("Run number")
plt.ylabel("Outcome")
ax = plt.gca()
ax.set_facecolor('#efeeea')
plt.bar(range(len(outcomes)), outcomes, color="#0A047A", width=1.0)
plt.show()

Q-table before training:
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0

RecursionError: maximum recursion depth exceeded

In [ ]:
#import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
plt.rcParams.update({'font.size': 17})

# We re-initialize the Q-table
qtable = np.zeros((64, 5))

# Hyperparameters
episodes = 1000        # Total number of episodes
alpha = 0.5            # Learning rate
gamma = 0.9            # Discount factor

# List of outcomes to plot
outcomes = []

print('Q-table before training:')
print(qtable)

# Training
for _ in range(episodes):
    state = env.reset()
    done = False

    # By default, we consider our outcome to be a failure
    outcomes.append("Failure")

    # Until the agent gets stuck in a hole or reaches the goal, keep training it
    while not done:
        # Choose the action with the highest value in the current state
        if np.max(qtable[state]) > 0:
            action = np.argmax(qtable[state])

        # If there's no best action (only zeros), take a random one
        else:
            action = env.action_space.sample()
             
        # Implement this action and move the agent in the desired direction
        new_state, reward, done, info = env.step(action)

        # Update Q(s,a)
        qtable[state, action] = qtable[state, action] + \
                                alpha * (reward + gamma * np.max(qtable[new_state]) - qtable[state, action])
        
        # Update our current state
        state = new_state

        # If we have a reward, it means that our outcome is a success
        if reward:
            outcomes[-1] = "Success"

print()
print('===========================================')
print('Q-table after training:')
print(qtable)

# Plot outcomes
plt.figure(figsize=(12, 5))
plt.xlabel("Run number")
plt.ylabel("Outcome")
ax = plt.gca()
ax.set_facecolor('#efeeea')
plt.bar(range(len(outcomes)), outcomes, color="#0A047A", width=1.0)
plt.show()

In [ ]:
episodes = 1000
nb_success = 0

# Evaluation
for _ in range(1000):
    state = env.reset()
    done = False
    
    # Until the agent gets stuck or reaches the goal, keep training it
    while not done:
        # Choose the action with the highest value in the current state
        if np.max(qtable[state]) > 0:
            action = np.argmax(qtable[state])

        # If there's no best action (only zeros), take a random one
        else:
            action = env.action_space.sample()
             
        new_state, reward, done, info = env.step(action)

        # Update our current state
        state = new_state

        # If we have a reward, it means that our outcome is a success
        nb_success += reward

print (f"Success rate = {nb_success/episodes*100}%")

In [15]:
import sys
print(sys.getrecursionlimit())

3000


In [ ]:
from IPython.display import clear_output
import time 

state = env.reset()
done = False
sequence = []

while not done:
    # Choose the action with the highest value in the current state
    if np.max(qtable[state]) > 0:
        action = np.argmax(qtable[state])

    # If there's no best action (only zeros), take a random one
    else:
        action = env.action_space.sample()
    
    # Add the action to the sequence
    sequence.append(action)

    # Implement this action and move the agent in the desired direction
    new_state, reward, done, info = env.step(action)

    # Update our current state
    state = new_state

    # Update the render
    clear_output(wait=True)
    env.render()
    time.sleep(1)

print(f"Sequence = {sequence}")